In [3]:
from scholarly import scholarly
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import numpy as np
from scholarly import scholarly
from typing import Union
from scholarly import ProxyGenerator

warnings.filterwarnings("ignore")
%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [90]:
import Levenshtein


def levenshtein_similarity(str1, str2):
    distance = Levenshtein.distance(str1, str2)
    max_len = max(len(str1), len(str2))
    similarity = 1 - (distance / max_len)
    return similarity


# Example usage


def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    similarity = len(intersection) / len(union)
    return similarity


# Example usage
t1 = "A. Haroon Akram-Lodhi"
t2 = "Haroon"
similarity = levenshtein_similarity(t1, t2)
print(f"Levenshtein Similarity: {similarity}")

similarity = jaccard_similarity(t1, t2)
print(f"Jaccard Similarity: {similarity}")

Levenshtein Similarity: 0.2857142857142857
Jaccard Similarity: 0.3333333333333333


In [185]:
import requests



In [324]:
coauthors = []
family_name = "edenhofer"  # "jiang"  # "mignone"  # "heuberger"  # "williams" #"lee"   # "teletzke"  #  "kheshgi"  # "calvin"
given_name = "ottmar"  # "bryan"  # "clara"  # "ellen" #"arthur"    ##"gary"  #  "haroon"  # "katherine"

base_url = "https://api.crossref.org/works"
start_year = 2004
end_year = 2024
# Parameters for the API request
params = {
    "query.author": given_name + " " + family_name,
    "filter": f"from-pub-date:{start_year}-01-01,until-pub-date:{end_year}-12-31",
    "rows": 1000,  # Number of results per page (max is 1000)
}

coauthors_set = set()  # To store unique co-authors


response = requests.get(base_url, params=params)
response.raise_for_status()  # Check for request errors
pub_list = response.json()["message"]["items"]
for pub in pub_list:
    authors = pub["author"]
    keep = True
    for a in authors:
        if "family" in a:
            if given_name in a["family"].lower():
                keep = False
            elif "given" in a:
                if (family_name in a["family"].lower()) and (
                    given_name not in a["given"].lower()
                ):
                    keep = False
                elif (family_name not in a["family"].lower()) and (
                    given_name in a["given"].lower()
                ):
                    keep = False
                elif family_name in a["given"].lower():
                    keep = False
                elif (a["family"].lower() == family_name) and (
                    given_name not in a["given"].lower()
                ):
                    keep = False
                elif (given_name in a["given"].lower()) and (
                    family_name not in a["family"].lower()
                ):
                    keep = False
    if keep:
        for author in authors:
            row_dict = {}
            row_dict["title"] = ""
            if "title" in pub:
                title = pub["title"][0]
            row_dict["title"] = title
            row_dict["ccs"] = 0
            for t in [
                "ccus",
                "ccs",
                "carbon capture",
                "carbon dioxide removal",
                "sequestration",
                "hydrogen",
                "carbon dioxide transport",
                "carbon dioxide capture",
                "co2 storage",
                "co2 transport",
            ]:
                if t in title.lower():
                    row_dict["ccs"] = 1
            row_dict["year"] = pub["published"]["date-parts"][0][0]
            row_dict["type"] = ""
            row_dict["doi"] = ""
            if "DOI" in pub:
                row_dict["doi"] = pub["DOI"]
            if "type" in pub:
                row_dict["type"] = pub["type"]
            row_dict["publisher"] = pub["publisher"]
            if "family" in author:
                if "given" in author:
                    author_full_name = (
                        author["given"].split(" ")[0] + " " + author["family"]
                    )
                else:
                    author_full_name = author["family"]
            else:
                author_full_name = author["name"]
            author_full_name = author_full_name.replace("‐", "-").replace(".", "")
            row_dict["author"] = author_full_name.title()
            row_dict["affiliation"] = ""
            if len(author["affiliation"]) > 0:
                row_dict["affiliation"] = author["affiliation"][0]["name"]
            coauthors.append(row_dict)

df = pd.DataFrame(coauthors)

In [325]:
df.ccs.value_counts()

ccs
0    1052
1      50
Name: count, dtype: int64

In [326]:
df[["title", "author", "ccs"]].groupby("title").agg({"ccs": "max"}).ccs.value_counts()

ccs
0    259
1     10
Name: count, dtype: int64

In [329]:
tmp = (
    df[["title", "doi", "year", "ccs"]]
    .groupby("title")
    .agg({"ccs": "max", "doi": "first", "year": "first"})
)  # .ccs.value_counts()
tmp.loc[tmp.ccs == 1]

,ccs,doi,year
title,,,
CCS-Bonds as a superior instrument to incentivize secure carbon sequestration,1,10.1016/j.egypro.2009.02.275,2009
"Comparison of carbon capture and storage with renewable energy technologies regarding structural, economic, and ecological aspects in Germany",1,10.1016/s1750-5836(07)00024-2,2007
Is atmospheric carbon dioxide removal a game changer for climate change mitigation?,1,10.1007/s10584-012-0681-4,2013
Low-stabilisation scenarios and technologies for carbon capture and sequestration,1,10.1016/j.egypro.2009.02.209,2009
On the Governance of Carbon Dioxide Removal – A Public Economics Perspective,1,10.1628/fa-2023-0012,2024
On the Governance of Carbon Dioxide Removal – a Public Economics Perspective,1,10.2139/ssrn.4422845,2023
Pigou's Advice and Sisyphus’Warning: Carbon Pricing Withnon-Permanent Carbon Dioxide Removal,1,10.2139/ssrn.4828800,2024
Sequencing Carbon Dioxide Removal into the EU ETS,1,10.2139/ssrn.4875550,2024
The Role of Carbon Capture and Sequestration Policies for Climate Change Mitigation,1,10.1007/s10640-013-9757-5,2014


In [328]:
df.groupby("author").agg(
    {
        "ccs": "sum",
        "title": "count",
        "year": "mean",
        "affiliation": lambda x: list(set(x)),
    }
).sort_values(by="title", ascending=False).head(60)

,ccs,title,year,affiliation
author,,,,
Ottmar Edenhofer,11,280,2015.914286,"[, Potsdam Institute for Climate Impact Resear..."
Michael Jakob,0,68,2017.544118,"[, Mercator Research Institute on Global Commo..."
Kai Lessmann,3,34,2014.647059,"[, * Potsdam Institute for Climate Impact Rese..."
Gunnar Luderer,1,30,2014.033333,"[, Potsdam Institute for Climate Impact Resear..."
Matthias Kalkuhl,5,27,2017.444444,"[, Mercator Research Institute on Global Commo..."
Nico Bauer,1,25,2012.760000,"[, * Potsdam Institute for Climate Impact Rese..."
Christian Flachsland,0,24,2014.541667,"[, Mercator Research Institute on Global Commo..."
Brigitte Knopf,0,24,2013.500000,"[, Potsdam Institute for Climate Impact Resear..."
Linus Mattauch,0,19,2016.315789,"[, Mercator Research Institute on Global Commo..."


In [154]:
authors = list(df.author.unique())
for a in authors:
    print(a)

Haroon S Kheshgi
Madhu Khanna
Haroon Kheshgi
Roger C Prince
David E Archer
Heleen De Coninck
John Kessels
Sergey Paltsev
Jennifer Morris
Howard Herzog
Steven J Smith
James A Edmonds
Shijie Shu
Atul K Jain
Michael Schlesinger
Joel Smith
John M Reilly
Tom Wilson
Matteo Muratori
Bryan Mignone
Haewon Mcjeon
Leon Clarke
Jae Edmonds
Karin Ritter
Robert Siveter
Miriam Lev-On
Theresa Shires
Angelo Gurgel
Bryan K Mignone
Steve Crookshank
Paulo Cunha
Arthur Lee
Leonard Bernstein
Erin Smith
Gary Teletzke
Jessica Farrell
Hans Thomann
Yang Song
William Landuyt
Felipe Feijoo
Corinne Hartin
Tzu-Shun Lin
Peter Lawrence
Atul Jain
Xiaojuan Yang
A David Mcguire
Wilfred Post
David Kicklighter
Oddvar Levang
Monica Linhares
Jaime Juez
Brigitte Poot
Charles J Vörösmarty
Henry Chen
Matthew Cervarich
Ximing Cai
Jaime Martin Juez
Page Kyle
Robert Link
Kali Benavides
Angelo C Gurgel
Bryan Chapman
Y-H Henry Chen
Matthew Mowers
Daniel Steinberg
Jordan Kearns
Jeffrey Palmer
Yen-Heng Henry Chen
Fredde Cappelen
Alain

In [57]:
for k in pub_list[40].keys():
    print(k)
    if isinstance(pub_list[40][k], dict):
        keys = pub_list[40][k].keys()
        for kk in keys:
            print(f" - {kk}")

indexed
 - date-parts
 - date-time
 - timestamp
reference-count
publisher
license
content-domain
 - domain
 - crossmark-restriction
short-container-title
published-print
 - date-parts
DOI
type
created
 - date-parts
 - date-time
 - timestamp
page
update-policy
source
is-referenced-by-count
special_numbering
title
prefix
volume
author
member
reference
container-title
language
link
deposited
 - date-parts
 - date-time
 - timestamp
score
resource
 - primary
issued
 - date-parts
references-count
alternative-id
URL
ISSN
issn-type
published
 - date-parts
assertion
article-number
